In [2]:
from urllib.request import urlopen
import gender_guesser.detector as gender
import pandas as pd
import requests
import json
import re
import csv

In [3]:
path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/goya.csv'

df_goya = pd.read_csv(path)

list_of_names_goya = df_goya['Movie Director'].tolist()
pattern = re.compile(r'(\d)') 
list_of_cleaned_names_goyas =[]
for name in list_of_names_goya:
    name_toclean = str(name)
    cleaned_strings = pattern.sub('', name_toclean)   
    list_of_cleaned_names_goyas.append(cleaned_strings.replace("  (no se otorgó)","").replace(" () ", "").replace('\n',''))
pattern = r'^\W*\w+'
list_of_names_final_goya = []
for director in list_of_cleaned_names_goyas:
    list_of_names_final_goya.append(re.findall(pattern, director))

# Imprimir los nombres limpios
list_of_cleaned_names_goya=[]
for name in list_of_names_final_goya:
    name_toclean = str(name)
    list_of_cleaned_names_goya.append(name_toclean
                                 .replace("['", "")
                                 .replace("']", "")
                                 .replace("nan", "")
                                 .replace("é", "e")
                                 .replace("Á", "A")
                                 .replace("í", "i"))
df_list_of_cleaned_names_goya = pd.DataFrame()
df_list_of_cleaned_names_goya['Name'] = list_of_cleaned_names_goya

In [4]:
# Function to detect gender from names
def detect_gender(df_list_of_cleaned_names_gender_goya, list_of_cleaned_names_goya):
    df_list_of_cleaned_names_gender_goya = pd.DataFrame()
    d = gender.Detector()
    
    df_list_of_cleaned_names_gender_goya['Gender'] = [d.get_gender(x) for x in list_of_cleaned_names_goya]
    df_list_of_cleaned_names_gender_goya.loc[ df_list_of_cleaned_names_gender_goya['Gender'] == 'mostly_male', 'Gender'] = 'male'
    df_list_of_cleaned_names_gender_goya.loc[ df_list_of_cleaned_names_gender_goya['Gender'] == 'mostly_female', 'Gender'] = 'female'
    df_results = pd.DataFrame()   
    df_results['Name'] = list_of_cleaned_names_goya
    df_results['Gender'] = df_list_of_cleaned_names_gender_goya
    return df_results

In [5]:
df_list_of_cleaned_names_gender_goya = detect_gender(df_list_of_cleaned_names_goya, list_of_cleaned_names_goya)

In [6]:
df_list_of_cleaned_names_gender_goya.tail(38)

,Name,Gender
1,Jose,male
2,Gonzalo,male
3,Ferdo,male
4,Carlos,male
5,Vicente,male
6,Ferdo,male
7,Luis,male
8,Imanol,male
9,Alex,male
10,Pilar,female


In [7]:
df_list_of_cleaned_names_gender_goya['Gender'].tail(38).value_counts().reset_index()

,Gender,count
0,male,33
1,female,4
2,unknown,1


In [8]:
path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/oscar.csv'

df_oscar = pd.read_csv(path)
directors_oscars = df_oscar['Movie Director'].tolist()
pattern = r'^\W*\w+'
list_of_names_oscars = []
for director in directors_oscars:
    list_of_names_oscars.append(re.findall(pattern, director))

list_of_cleaned_names_oscars=[]
unicode_character = "\u0161"
df_names = pd.DataFrame()
for name in list_of_names_oscars:
    name_toclean = str(name)
    sin_milos = name_toclean.replace('Miloš','Milos')
    list_of_cleaned_names_oscars.append(sin_milos.replace("['", "").replace("']", "").replace("é","e"))

df_list_of_cleaned_names_oscars = pd.DataFrame()
df_list_of_cleaned_names_oscars['Name'] = list_of_cleaned_names_oscars

In [9]:
def detect_gender(df, list_of_names):
    # Make a copy of the DataFrame to avoid modifying the original DataFrame
    df_result = df.copy()   
    # Initialize a gender detector
    d = gender.Detector()   
    # Create a list to store gender predictions based on names
    gender_predictions = []   
    # Iterate over each name in the list_of_names and predict the gender
    for name in list_of_names:
        gender_predictions.append(d.get_gender(name))
    
    # Map 'mostly_male' to 'male' and 'mostly_female' to 'female'
    gender_predictions = ['male' if gender == 'mostly_male' else
                          'female' if gender == 'mostly_female' else
                          'unknown' if gender == 'unknown' else
                          gender for gender in gender_predictions] 
    df_results = pd.DataFrame()
    df_results['Name'] = df
    df_results['Gender'] = gender_predictions
    return df_results

In [10]:
gender_predictions_oscars = detect_gender(df_list_of_cleaned_names_oscars, list_of_cleaned_names_oscars)
gender_predictions_oscars.tail(38)

,Name,Gender
58,Oliver,male
59,Bernardo,male
60,Barry,male
61,Bruce,male
62,Kevin,male
63,Jonathan,male
64,Clint,male
65,Steven,male
66,Robert,male
67,Mel,female


In [11]:
gender_predictions_oscars['Gender'].tail(38).value_counts().reset_index()

,Gender,count
0,male,33
1,female,4
2,andy,1


##### Comparando los resultados:
###### . Las mujeres segun el codigo en goya seria = 4 y yo veo = 4
###### . Las mujeres segun el codigo en oscars seria = 10 y yo veo = 3

#### Verificando por mi mismo

In [12]:
path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/goya_gender.csv'

df_goya = pd.read_csv(path,sep=';')
df_goya

,Goya Year,Movie Director,Gender
0,1.ª edición - 1987 17 de marzo de 1987,Fernando Fernán Gómez (no se otorgó),Male
1,2.ª edición - 1988 22 de marzo de 1988,José Luis Garci (no se otorgó),Male
2,3.ª edición - 1989 22 de marzo de 1989,Gonzalo Suárez (no se otorgó),Male
3,4.ª edición - 1990 10 de marzo de 1990,Fernando Trueba (1) Ana Díez,Male
4,5.ª edición - 1991 16 de febrero de 1991,Carlos Saura Rosa Vergés,Male
5,6.ª edición - 1992 7 de marzo de 1992,Vicente Aranda Juanma Bajo Ulloa,Male
6,7.ª edición - 1993 14 de marzo de 1993,Fernando Trueba (2) Julio Medem,Male
7,8.ª edición - 1994 21 de enero de 1994,Luis García Berlanga Mariano Barroso,Male
8,9.ª edición - 1995 20 de enero de 1995,Imanol Uribe La Cuadrilla (S. Aguilar y L. Gu...,Male
9,10.ª edición - 1996 27 de enero de 1996,Álex de la Iglesia Agustín Díaz Yanes,Male


In [13]:
path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/oscar_gender.csv'

df_oscar = pd.read_csv(path,sep=';')
df_oscar

,Oscar Year,Movie Title,Movie Studio,Movie Productor,Movie Director,Gender,Movie Genre
0,1.ª ceremonia de entrega de los Premios Óscar ...,Wings,Paramount Famous Players-Lasky,Lucien Hubbard,William A. Wellman,Male,Bélico / Cine mudo
1,2.ª ceremonia de entrega de los Premios Óscar ...,The Broadway Melody,Metro-Goldwyn-Mayer,Irving ThalbergLawrence Weingarten,Harry Beaumont,Male,Musical
2,3.ª ceremonia de entrega de los Premios Óscar ...,All Quiet on the Western Front,Universal,Carl Laemmle Jr.,Lewis Milestone,Male,Bélico
3,4.ª ceremonia de entrega de los Premios Óscar ...,Cimarron,RKO Radio,William LeBaron,Wesley Ruggles,Male,Western
4,5.ª ceremonia de entrega de los Premios Óscar ...,Grand Hotel,Metro-Goldwyn-Mayer,Irving Thalberg,Edmund Goulding,Male,Drama
...,...,...,...,...,...,...,...
91,92.ª ceremonia de entrega de los Premios Óscar...,기생충 - Gisaengchung,Barunson E&A Corp,Kwak Sin-ae Bong Joon-ho,Bong Joon-ho,Male,Drama / Suspenso / Humor negro
92,93.ª ceremonia de entrega de los Premios Óscar...,Nomadland,Searchlight Pictures,Frances McDormandPeter SpearsMollye AsherDan J...,Chloé Zhao,Female,Drama
93,94.ª ceremonia de entrega de los Premios Óscar...,CODA,PathéApple TV+,Philippe RousseletFabrice GianfermiPatrick Wac...,Sian Heder,Female,Comedia dramática / Coming-of-age
94,95.ª ceremonia de entrega de los Premios Óscar...,Everything Everywhere All at Once,A24IAC FilmsGozie AGBOYear of the RatLey Line ...,Daniel KwanDaniel ScheinertJonathan Wang,Daniel Kwan,Male,Acción / Comedia dramática / Ciencia ficción


In [14]:
path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/goya_gender.csv'

df_goya = pd.read_csv(path,sep=';')
print("Premios Goya")
df_goya['Gender'].tail(38).value_counts().reset_index() 

Premios Goya


,Gender,count
0,Male,34
1,Female,4


In [15]:
df_goya.loc[df_goya['Gender']=='Female']

,Goya Year,Movie Director,Gender
10,11.ª edición - 1997 25 de enero de 1997,Pilar Miró Alejandro Amenábar (1),Female
17,18.ª edición - 2004 31 de enero de 2004,Icíar Bollaín Ángeles González-Sinde,Female
19,20.ª edición - 2006 29 de enero de 2006,Isabel Coixet (1) José Corbacho y Juan Cruz,Female
31,32.ª edición - 2018 3 de febrero de 2018,Isabel Coixet (2) Carla Simón,Female


In [16]:
path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/oscar_gender.csv'

# path = 'C:/Users/user/Jupyter/IronHack/Work - Python/Week3/Personal Project Web Scraping/oscar.csv'

df_oscar = pd.read_csv(path,sep=';')
print("Premios Oscar")
df_oscar['Gender'].tail(38).value_counts().reset_index()

Premios Oscar


,Gender,count
0,Male,35
1,Female,3


In [17]:
df_oscar.loc[df_oscar['Gender']=='Female']

,Oscar Year,Movie Title,Movie Studio,Movie Productor,Movie Director,Gender,Movie Genre
81,82.ª ceremonia de entrega de los Premios Óscar...,The Hurt Locker,Summit Entertainment,Kathryn BigelowMark Boal Nicolas Chartier Greg...,Kathryn Bigelow,Female,Acción / Suspense
92,93.ª ceremonia de entrega de los Premios Óscar...,Nomadland,Searchlight Pictures,Frances McDormandPeter SpearsMollye AsherDan J...,Chloé Zhao,Female,Drama
93,94.ª ceremonia de entrega de los Premios Óscar...,CODA,PathéApple TV+,Philippe RousseletFabrice GianfermiPatrick Wac...,Sian Heder,Female,Comedia dramática / Coming-of-age


In [21]:
df_goya['Gender'].tail(10).value_counts().reset_index() 

,Gender,count
0,Male,9
1,Female,1


In [23]:
df_oscar['Gender'].tail(10).value_counts().reset_index()

,Gender,count
0,Male,8
1,Female,2
